<a href="https://colab.research.google.com/github/rodmart21/Sports_analytics/blob/main/webscraping_football_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import requests

from bs4 import BeautifulSoup

In [2]:
# Scrape a player page
url = "https://www.transfermarkt.us/erling-haaland/profil/spieler/418560"
player_id = url.split('/')[-1]

In [3]:
# Get headers from https://www.whatismybrowser.com/detect/what-http-headers-is-my-browser-sending
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

In [4]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}

In [5]:
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
# BeautifulSoup(): Convierte el contenido HTML en un objeto BeautifulSoup,
# lo que facilita la manipulación y el análisis del HTML. Esto es útil para extraer datos específicos de la página web, como títulos, párrafos, enlaces, tablas, etc.

In [6]:
response.status_code

200

In [7]:
p=soup.select_one('h1[class="data-header__headline-wrapper"]')  # Lo cogemos con el inspect de la página de Transfermarkt.
                                                                # Haces click derecho y la última opción

In [8]:
print(p)

<h1 class="data-header__headline-wrapper">
<span class="data-header__shirt-number">
                        #9                    </span>
                                Erling <strong>Haaland</strong> </h1>


In [9]:
player_name = soup.select_one('h1[class="data-header__headline-wrapper"]').text.split('\n')[-1].strip()
player_number = soup.select_one('span[class="data-header__shirt-number"]').text.strip().replace('#', '')

In [16]:
equipo = soup.select_one('h1[class="data-header__info-box"]')
print(equipo)

None


In [ ]:
equipo = soup.select_one('h1[class="data-header__club"]')
print(equipo)

In [14]:
player_name = soup.select_one('h1[class="data-header__headline-wrapper"]')
print(player_name)

<h1 class="data-header__headline-wrapper">
<span class="data-header__shirt-number">
                        #9                    </span>
                                Erling <strong>Haaland</strong> </h1>


## Hay webs en las que es más sencillo extraer los datos, Transfermarkt no es una de ellas, así que se puede seguir otra estrategia.

### En castellano, en regex101 lo detecta, pero fuera de ahí tienes que escribirlo como en la web original (en inglés).


In [38]:
a=re.search("Contract expires: .*__content\">(.*?)</span>",str(soup)).group(1)  #Este es el primer regex que se le ocurrió, se puede mejorar.
print(a)

Jun 30, 2027


In [39]:
a=re.search("Contrato hasta .*__content\">(.*?)</span>",str(soup)).group(1)
print(a)

AttributeError: 'NoneType' object has no attribute 'group'

In [21]:
# Let's use regex to find the player's contract expiry date
player_contract_expiry = re.search("Contract expires: (.*)", soup.text).group(1)

# Now let's add some more complex regex patterns to find the player's birthplace
player_birthplace = re.search("Place of birth:.*?([A-z].*?) ", soup.text, re.DOTALL).group(1).strip()

# We can reuse both of those ideas to find other player information but just replacing the name

# We might have to switch a couple of things though in the regex pattern like accounting for two words
player_agent = re.search("Agent:.*?([A-z].*?)\n", soup.text, re.DOTALL).group(1).strip()

# We can also use regex to find the player's height which has numbers a comma and letters
player_height = re.search("Height:.*?([0-9].*?)\n", soup.text, re.DOTALL).group(1).strip()


In [23]:
player_contract_expiry

'Jun 30, 2027'

## La tercera forma. Con la llamada a la API y el código que cogemos de https://curlconverter.com/ dónde solo nos quedamos con la última parte que es la que nos interesa.

In [40]:
response=requests.get('https://www.transfermarkt.es/ceapi/marketValueDevelopment/graph/418560',cookies=cookies,headers=headers,)

NameError: name 'cookies' is not defined

In [47]:
player_id

'418560'

In [44]:
# Añadimos la f para decir que es un string, y también el player_id.
response=requests.get(f'https://www.transfermarkt.es/ceapi/marketValueDevelopment/graph/{player_id}', headers=headers,)

In [45]:
response.status_code  # 200 significa que ha ido bien, 404 por ejemplo, que ha ido mal.

200

In [49]:
response.json()  # Tenemos la lista con todos sus valores de mercado.

In [52]:
response.json().keys()

dict_keys(['list', 'current', 'highest', 'highest_date', 'last_change', 'details_url', 'thread', 'translations'])

In [7]:
# Ok so now that we've got some basic regex patterns, let's move on to using api endpoints
# curlconverter.com
api_endpoints = [
    f"marketValueDevelopment/graph/{player_id}",
    f"transferHistory/list/{player_id}",
    f"player/{player_id}/performance"
]

for endpoint in api_endpoints:
    response = requests.get(
        'https://www.transfermarkt.us/ceapi/' + endpoint,
        headers=headers
    ).json()

## Scrapping data form SofaScore. I add the football from Real Madrid

In [55]:
import requests
from bs4 import BeautifulSoup

In [ ]:
https://www.sofascore.com/es/real-madrid-villarreal/ugbsEgb#id:11368619,tab:details

In [56]:
response = requests.get(
    'https://www.sofascore.com/arsenal-manchester-city/rsR#10385636',
    headers={'User-Agent': 'Mozilla/5.0'} # you'll be blocked if you don't use some type of user agent
)

In [59]:
response = requests.get(
    'https://www.sofascore.com/es/real-madrid-villarreal/ugbsEgb#id:11368619,tab:details')

In [60]:
response.status_code

200

### Los headers los incuimos de la página: https://www.whatismybrowser.com/detect/what-http-headers-is-my-browser-sending , igual que hacíamos antes. Normalmente aparece algo más que lo de Mozilla, pero en este caso con eso es necesario, lo importante es que el status_code sea 200.



In [61]:
response = requests.get(
    'https://www.sofascore.com/es/real-madrid-villarreal/ugbsEgb#id:11368619,tab:details',
    headers={'User-Agent': 'Mozilla/5.0'} # you'll be blocked if you don't use some type of user agent.
)

In [62]:
response.status_code  # 200 está bien

200

## Crear una instancia de BeautifulSoup para analizar el HTML


In [63]:
soup = BeautifulSoup(response.text, 'html.parser')

If we wanted to scrape all of the shots using just normal requests and beautiful soup...
We'd have to do some pretty complicated stuff to get the data we want

In [65]:
soup.select('g[cursor="pointer"]'  # No lo podemos cargar, el código viene de JavaScript. Debemos usar la opción de la API.

[]

In [64]:
# As you can see we can't even get the shots because they are dynamically loaded
soup.select('g[cursor="pointer"]')

[]

What we'll do is use the API's they are loading to call that data directly

If we look in the `network` tab of the developer tools we can see the API calls they are making
and then we can copy that and make it ourselves in our code

# Steps:
1. Find the API call `shotmap` in the network tab
2. Right click and copy as a cURL
3. go to curlconverter.com and paste the cURL
4. copy the python code

In [66]:
# Lo que sacamos del curlconverter.x

import requests

cookies = {
    '_gcl_au': '1.1.2081927633.1716149495',
    '_ga': 'GA1.1.1941725413.1716149497',
    'FCCDCF': '%5Bnull%2Cnull%2Cnull%2C%5B%22CP-3IoAP-3IoAEsACBENA1EoAP_gAEPgAA6II3gB5C5ETSFBYH51KIsUYAEHwAAAIsAgAAYBAQABQBKQAIQCAGAAEAhAhCACgAAAIEYBIAEACAAQAAAAAAAAIAAEIAAQAAAIICAAAAAAAABIAAAIAAAAEAAAwCAABAAA0AgEAJIISMgAAAAAAAAAAgAAAAAAAgAAAEhAAAEIAAAAACgAEABAEAAAAAEIABBII3gB5C5ETSFBYHhVIIMUIAERQAAAIsAgAAQBAQAAQBKQAIQCEGAAAAgAACAAAAAAIEQBIAEAAAgAAAAAAAAAIAAEAAAAAAAIICAAAAAAAABAAAAIAAAAAAAAwCAABAAAwQhEAJIASEgAAAAgAAAAAoAAAAAAAgAAAEhAAAEAAAAAAAAAEAAAEAAAAAAAABBIAAA.dnAACAgAAAA%22%2C%222~41.70.89.108.149.211.313.358.415.486.540.621.981.1029.1046.1092.1097.1126.1205.1301.1516.1558.1584.1598.1651.1697.1716.1753.1810.1832.1985.2328.2373.2440.2571.2572.2575.2577.2628.2642.2677.2767.2860.2878.2887.2922.3182.3190.3234.3290.3292.3331.10631~dv.%22%2C%22D59998C3-B901-4C02-BBB2-BDDFE43E2F71%22%5D%5D',
    '__qca': 'P0-256114773-1716149503180',
    'cto_bundle': 'FoUvXF82R2hvSGRab0ZDaDA2cVBJalB4dm8zNlpJb2VROWw4aW1SaUhSa1ZmQURlV3FuYkd0MGxLT096QlZ6U1pQNjZXS0RBbjRkU2xJeHpyUzBuaFhQYnRuaFBmamFTVHBTN2ZTd0lvTkp2eks5NFRzQ2MyY096aFRPUGgwYmxoRWd0dGU3Um1QZ3ViNnpGWTZZNmlncWw4V21xbXhPSzlJTXFmOWV4RXJvVjdLU1klM0Q',
    '__gads': 'ID=5d75f2defa542b67:T=1716149503:RT=1716151368:S=ALNI_MZw9Ix0nE-Z5ANRGCYWN9kD7mdITQ',
    '__gpi': 'UID=00000d77dc0a78c4:T=1716149503:RT=1716151368:S=ALNI_MYOwWy2DfRrGHLHTg67ZGbALi6DRw',
    '__eoi': 'ID=f8c69ad5d57acdd5:T=1716149504:RT=1716151368:S=AA-AfjbGVbAMiWc-xXla9msMNOPW',
    'FCNEC': '%5B%5B%22AKsRol8v_YosFsb-eEuh-Off_X3IiDVmt-cjmf-Kg37lmygsEGbrPYoY50QH3lcrTiu-R0I6uJ8cHB1uaxdu1rnIIQYUW4xbhbodJMZmXKNgDif0Mii9m2Nzkpg9OcBe9HQYJ414g5uoNcDvaXmMieRjOOFm2aRacw%3D%3D%22%5D%5D',
    '_ga_QH2YGS7BB4': 'GS1.1.1716149496.1.1.1716151386.0.0.0',
    '_ga_3KF4XTPHC4': 'GS1.1.1716149496.1.1.1716151386.57.0.0',
    '_ga_HNQ9P9MGZR': 'GS1.1.1716149498.1.1.1716151398.54.0.0',
}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    # 'cookie': '_gcl_au=1.1.2081927633.1716149495; _ga=GA1.1.1941725413.1716149497; FCCDCF=%5Bnull%2Cnull%2Cnull%2C%5B%22CP-3IoAP-3IoAEsACBENA1EoAP_gAEPgAA6II3gB5C5ETSFBYH51KIsUYAEHwAAAIsAgAAYBAQABQBKQAIQCAGAAEAhAhCACgAAAIEYBIAEACAAQAAAAAAAAIAAEIAAQAAAIICAAAAAAAABIAAAIAAAAEAAAwCAABAAA0AgEAJIISMgAAAAAAAAAAgAAAAAAAgAAAEhAAAEIAAAAACgAEABAEAAAAAEIABBII3gB5C5ETSFBYHhVIIMUIAERQAAAIsAgAAQBAQAAQBKQAIQCEGAAAAgAACAAAAAAIEQBIAEAAAgAAAAAAAAAIAAEAAAAAAAIICAAAAAAAABAAAAIAAAAAAAAwCAABAAAwQhEAJIASEgAAAAgAAAAAoAAAAAAAgAAAEhAAAEAAAAAAAAAEAAAEAAAAAAAABBIAAA.dnAACAgAAAA%22%2C%222~41.70.89.108.149.211.313.358.415.486.540.621.981.1029.1046.1092.1097.1126.1205.1301.1516.1558.1584.1598.1651.1697.1716.1753.1810.1832.1985.2328.2373.2440.2571.2572.2575.2577.2628.2642.2677.2767.2860.2878.2887.2922.3182.3190.3234.3290.3292.3331.10631~dv.%22%2C%22D59998C3-B901-4C02-BBB2-BDDFE43E2F71%22%5D%5D; __qca=P0-256114773-1716149503180; cto_bundle=FoUvXF82R2hvSGRab0ZDaDA2cVBJalB4dm8zNlpJb2VROWw4aW1SaUhSa1ZmQURlV3FuYkd0MGxLT096QlZ6U1pQNjZXS0RBbjRkU2xJeHpyUzBuaFhQYnRuaFBmamFTVHBTN2ZTd0lvTkp2eks5NFRzQ2MyY096aFRPUGgwYmxoRWd0dGU3Um1QZ3ViNnpGWTZZNmlncWw4V21xbXhPSzlJTXFmOWV4RXJvVjdLU1klM0Q; __gads=ID=5d75f2defa542b67:T=1716149503:RT=1716151368:S=ALNI_MZw9Ix0nE-Z5ANRGCYWN9kD7mdITQ; __gpi=UID=00000d77dc0a78c4:T=1716149503:RT=1716151368:S=ALNI_MYOwWy2DfRrGHLHTg67ZGbALi6DRw; __eoi=ID=f8c69ad5d57acdd5:T=1716149504:RT=1716151368:S=AA-AfjbGVbAMiWc-xXla9msMNOPW; FCNEC=%5B%5B%22AKsRol8v_YosFsb-eEuh-Off_X3IiDVmt-cjmf-Kg37lmygsEGbrPYoY50QH3lcrTiu-R0I6uJ8cHB1uaxdu1rnIIQYUW4xbhbodJMZmXKNgDif0Mii9m2Nzkpg9OcBe9HQYJ414g5uoNcDvaXmMieRjOOFm2aRacw%3D%3D%22%5D%5D; _ga_QH2YGS7BB4=GS1.1.1716149496.1.1.1716151386.0.0.0; _ga_3KF4XTPHC4=GS1.1.1716149496.1.1.1716151386.57.0.0; _ga_HNQ9P9MGZR=GS1.1.1716149498.1.1.1716151398.54.0.0',
    'if-none-match': 'W/"256303f320"',
    'priority': 'u=1, i',
    'referer': 'https://www.sofascore.com/es/real-madrid-villarreal/ugbsEgb',
    'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'x-requested-with': '337e31',
}

response = requests.get('https://www.sofascore.com/api/v1/event/11368619/shotmap', cookies=cookies, headers=headers)

In [67]:
response.status_code  # Obtenemos un 304,

304

## Añadir esta fecha a nuestro código, para evitar ese error

In [77]:
headers['If-Modified-Since'] = 'Tues, 18 Jul 2023 00:00:00 GMT'

In [78]:
response = requests.get('https://www.sofascore.com/api/v1/event/11368619/shotmap', headers=headers)
response

<Response [200]>

In [79]:
response.json()

{'shotmap': [{'player': {'name': 'Bertrand Traoré',
    'firstName': '',
    'lastName': '',
    'slug': 'bertrand-traore',
    'shortName': 'B. Traoré',
    'position': 'M',
    'jerseyNumber': '25',
    'userCount': 4868,
    'id': 218160,
    'fieldTranslations': {'nameTranslation': {'ar': 'برتران تراوري إيزيدور'},
     'shortNameTranslation': {'ar': 'ب. ت. إيزيدور'}}},
   'isHome': True,
   'shotType': 'save',
   'situation': 'assisted',
   'playerCoordinates': {'x': 10.6, 'y': 73, 'z': 0},
   'bodyPart': 'left-foot',
   'goalMouthLocation': 'low-right',
   'goalMouthCoordinates': {'x': 0, 'y': 46, 'z': 3.8},
   'blockCoordinates': {'x': 1.5, 'y': 54.2, 'z': 0},
   'xg': 0.03035057336092,
   'xgot': 0.1829,
   'id': 3262267,
   'time': 90,
   'timeSeconds': 5366,
   'draw': {'start': {'x': 73, 'y': 10.6},
    'block': {'x': 54.2, 'y': 1.5},
    'end': {'x': 54, 'y': 0},
    'goal': {'x': 54, 'y': 96.2}},
   'reversedPeriodTime': 1,
   'reversedPeriodTimeSeconds': 34,
   'incidentTy

### Con su ejemplo particular.

In [70]:
# These are the headers we need to access the API
headers = {
    'authority': 'api.sofascore.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'dnt': '1',
    'if-none-match': 'W/"4bebed6144"',
    'origin': 'https://www.sofascore.com',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

# you'll need to change the url to match the one you copied. As you can see they have some sort of match id to identify the game --> 10385636
response = requests.get('https://api.sofascore.com/api/v1/event/10385636/shotmap', headers=headers)

In [71]:
# If you make the request without doing anything else though you will get a 304 response
# which means "not modified"
response

<Response [200]>

In [72]:
# The way to fix this is to add the 'If-Modified-Since' header to the request
# You can change this dynamically or just hardcode and change it every time you run the script
headers['If-Modified-Since'] = 'Tues, 18 Jul 2023 00:00:00 GMT'

In [73]:
# Now we get a 200 response
response = requests.get('https://api.sofascore.com/api/v1/event/10385636/shotmap', headers=headers)
response

<Response [200]>

In [74]:
# Now we can get the data we want
shots = response.json()

### This is an example of one of the many ways to get data from sofascore,you can also look through their different API's and see what you can find